In [5]:
import pandas as pd
import math

import torch

from nltk.tokenize import word_tokenize
from sklearn.preprocessing import LabelEncoder
from torch.nn.utils.rnn import pad_sequence
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim

from string import punctuation

from tqdm import tqdm

import enchant
from enchant.tokenize import get_tokenizer
from enchant.tokenize import basic_tokenize

import pickle


In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [7]:
df = pd.read_csv('./dataset/ielts-writing-essays.csv')
df.head(5)

label_encoder = LabelEncoder()

In [8]:
essay = df['Essay'].to_list()

essays_token = []
all_token = []

for i in essay:
    token = []
    words = word_tokenize(i)
    for word in words:
        if word not in punctuation and word.isalpha():
            word = word.lower()
            all_token.append(word)
        
            token.append(word)
    essays_token.append(token)
all_vocab = list(set(all_token))
all_vocab = ['<unk>'] + all_vocab
label_encoder.fit(all_vocab)

with open("label_encoder.pkl", "wb") as file:
    pickle.dump(label_encoder, file)

print(f"Len Essays Token :{len(essays_token)}")
print(f"Len All Token :{len(all_token)}")
print(f"Len All Vocab :{len(all_vocab)}")
SEQ_LEN = 50

Len Essays Token :1435
Len All Token :359245
Len All Vocab :13350


In [9]:
def calculate_average_length(sentences):
    total_len = 0
    for sentence in sentences:
        total_len += len(sentence)
    average_len = int(round(total_len / len(sentences), 0))
    return average_len

In [10]:
def get_data(dataset, batch_size):
    """Convert tokenized dataset to tensor batches."""
    data = [torch.LongTensor(label_encoder.transform(tokens)) for tokens in dataset]
    data = pad_sequence(data, batch_first=True, padding_value=0)
    num_batches = data.shape[0] // batch_size
    data = data[:num_batches * batch_size]
    data = data.view(batch_size, -1) 
    return data

In [11]:
train_len = round(len(essays_token) * 0.7)
test_val_len = round(len(essays_token) * 0.15)

train_data = essays_token[:train_len]
test_data = essays_token[train_len: train_len + test_val_len]
val_data = essays_token[train_len + test_val_len:]
# 0.7 0.15

batch_size = 16
print(len(train_data))
print(len(test_data))

train_data = get_data(train_data,batch_size)
test_data = get_data(test_data,batch_size)
val_data = get_data(val_data,batch_size)

1004
215


In [12]:
print(train_data)

tensor([[ 1210,   553,     1,  ...,     0,     0,     0],
        [  103, 12133, 11974,  ...,     0,     0,     0],
        [12133,  7822,  5780,  ...,     0,     0,     0],
        ...,
        [12016, 12411,  5410,  ...,     0,     0,     0],
        [ 6625, 10862, 10804,  ...,     0,     0,     0],
        [12046,  5407, 10804,  ...,     0,     0,     0]])


In [13]:

class LSTM(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, num_layers, dropout_rate):
                
        super().__init__()
        self.num_layers = num_layers
        self.hidden_dim = hidden_dim
        self.embedding_dim = embedding_dim

        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers=num_layers, 
                    dropout=dropout_rate, batch_first=True)
        self.dropout = nn.Dropout(dropout_rate)
        self.fc = nn.Linear(hidden_dim, vocab_size)
        
        
        self.init_weights()

    def forward(self, src, hidden):
        embedding = self.dropout(self.embedding(src))
        output, hidden = self.lstm(embedding, hidden)          
        output = self.dropout(output) 
        prediction = self.fc(output)
        return prediction, hidden
    
    def init_weights(self):
        init_range_emb = 0.1
        init_range_other = 1/math.sqrt(self.hidden_dim)
        self.embedding.weight.data.uniform_(-init_range_emb, init_range_emb)
        self.fc.weight.data.uniform_(-init_range_other, init_range_other)
        self.fc.bias.data.zero_()
        for i in range(self.num_layers):
            self.lstm.all_weights[i][0] = torch.FloatTensor(self.embedding_dim,
                    self.hidden_dim).uniform_(-init_range_other, init_range_other) 
            self.lstm.all_weights[i][1] = torch.FloatTensor(self.hidden_dim, 
                    self.hidden_dim).uniform_(-init_range_other, init_range_other) 

    def init_hidden(self, batch_size, device):
        hidden = torch.zeros(self.num_layers, batch_size, self.hidden_dim).to(device)
        cell = torch.zeros(self.num_layers, batch_size, self.hidden_dim).to(device)
        return hidden, cell
    
    
    def detach_hidden(self, hidden):
        hidden, cell = hidden
        hidden = hidden.detach()
        cell = cell.detach()
        return hidden, cell

In [14]:
VOCAB_SIZE = len(all_vocab)
EMBED_DIM = 256
HIDDEN_DIM = 128       
NUM_LAYERS = 3   
DROPOUT_RATE = 0.75
lr = 1e-3
EPOCHS = 50
clip = 0.25


In [15]:

model = LSTM(VOCAB_SIZE, EMBED_DIM, HIDDEN_DIM, NUM_LAYERS, DROPOUT_RATE).to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'The model has {num_params:,} trainable parameters')

The model has 5,601,574 trainable parameters


In [16]:
def get_batch(data, seq_len, idx):
    src = data[:, idx:idx + seq_len]
    tgt = data[:, idx + 1:idx + seq_len + 1]
    return src, tgt

def train_epoch(model, data, optimizer, criterion, batch_size, seq_len, clip,device):
    model.train()
    epoch_loss = 0
    num_batches = data.shape[1] // seq_len
    hidden = model.init_hidden(batch_size,device)

    for idx in tqdm(range(0, num_batches * seq_len, seq_len), desc='Training', leave=False):
        src, tgt = get_batch(data, seq_len, idx)
        src, tgt = src.to(device), tgt.to(device)

        optimizer.zero_grad()
        hidden = model.detach_hidden(hidden)
        predictions, hidden = model(src, hidden)

        predictions = predictions.reshape(-1, VOCAB_SIZE)
        tgt = tgt.reshape(-1)
        loss = criterion(predictions, tgt)

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()

        epoch_loss += loss.item()

    return epoch_loss / num_batches

def evaluate_epoch(model, data, criterion, batch_size, seq_len,device):
    model.eval()
    epoch_loss = 0
    num_batches = data.shape[1] // seq_len
    hidden = model.init_hidden(batch_size,device)

    with torch.no_grad():
        for idx in range(0, num_batches * seq_len, seq_len):
            src, tgt = get_batch(data, seq_len, idx)
            src, tgt = src.to(device), tgt.to(device)

            hidden = model.detach_hidden(hidden)
            predictions, hidden = model(src, hidden)

            predictions = predictions.reshape(-1, VOCAB_SIZE)
            tgt = tgt.reshape(-1)
            loss = criterion(predictions, tgt)

            epoch_loss += loss.item()

    return epoch_loss / num_batches

In [17]:
lr_scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.5, patience=0)

best_valid_loss = float('inf')

for epoch in range(EPOCHS):
    train_loss = train_epoch(model, train_data, optimizer, criterion, batch_size, SEQ_LEN, clip,device)
    valid_loss = evaluate_epoch(model, val_data, criterion, batch_size, SEQ_LEN,device)

    lr_scheduler.step(valid_loss)

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'best-val-lstm_lm.pt')

    print(f"Epoch {epoch+1}:")
    print(f"Train Loss: {train_loss:.4f} | Train Perplexity: {math.exp(train_loss):.3f}")
    print(f"Valid Loss: {valid_loss:.4f} | Validation Perplexity: {math.exp(valid_loss):.3f}")

model.load_state_dict(torch.load('best-val-lstm_lm.pt'))
test_loss = evaluate_epoch(model, test_data, criterion, batch_size, SEQ_LEN,device)
print(f"Test Loss: {test_loss:.4f} | Test Perplexity: {math.exp(test_loss):.3f}")

torch.save(model, "entire_model.pth")



Epoch 1:
Train Loss: 3.7256 | Train Perplexity: 41.498
Valid Loss: 3.7468 | Validation Perplexity: 42.384


Epoch 2:
Train Loss: 2.8289 | Train Perplexity: 16.927
Valid Loss: 3.6988 | Validation Perplexity: 40.400


Epoch 3:
Train Loss: 2.7831 | Train Perplexity: 16.169
Valid Loss: 3.6737 | Validation Perplexity: 39.396


Epoch 4:
Train Loss: 2.7547 | Train Perplexity: 15.716
Valid Loss: 3.6574 | Validation Perplexity: 38.761


Epoch 5:
Train Loss: 2.7281 | Train Perplexity: 15.304
Valid Loss: 3.6435 | Validation Perplexity: 38.224


...

Epoch 48:
Train Loss: 2.4945 | Train Perplexity: 12.116
Valid Loss: 3.5420 | Validation Perplexity: 34.537


Epoch 49:
Train Loss: 2.4946 | Train Perplexity: 12.117
Valid Loss: 3.5420 | Validation Perplexity: 34.537


Epoch 50:
Train Loss: 2.4958 | Train Perplexity: 12.131
Valid Loss: 3.5420 | Validation Perplexity: 34.537


C:\Users\Win10\AppData\Local\Temp\ipykernel_19048\1174497199.py:19: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('best-val-lstm_lm.pt'))


Test Loss: 2.9507 | Test Perplexity: 19.120


In [18]:
def generate_text(model, prompt, seq_len, vocab, label_encoder, device, max_length=50):
    tokens = word_tokenize(prompt)
    token_indices = label_encoder.transform(tokens)
    
    input_tensor = torch.LongTensor(token_indices).unsqueeze(0).to(device)  
    
    hidden = model.init_hidden(1,device)
    
    # Generate text
    model.eval()
    generated_text = tokens.copy()  
    for _ in range(max_length):
        # Forward pass
        with torch.no_grad():
            predictions, hidden = model(input_tensor, hidden)
        predicted_idx = predictions[0, -1].argmax().item()
        if predicted_idx == 0:  
            continue
        predicted_word = label_encoder.inverse_transform([predicted_idx])[0]
        
        generated_text.append(predicted_word)
        
        input_tensor = torch.cat((input_tensor, torch.LongTensor([predicted_idx]).unsqueeze(0).to(device)), dim=1)
    
    generated_text_str = ' '.join(generated_text)
    return generated_text_str


In [ ]:
prompt = "school"
generated_text = generate_text(model, prompt, SEQ_LEN, all_vocab, label_encoder, device, max_length=100)
print(f"Generated text: {generated_text}")


Generated text: school and the other hand the number of the number of the number of the number of the number of the number of the number of the number of the number of the number of the number of the number of the number of the number of the number of the number of the number of the number of the number of the number of the number of the number of the number of the number of the number of the number of the number of the number of the number of the number of the number of the number of


: 